In [32]:
import pandas as pd
import numpy as np
import librosa
from joblib import Parallel, delayed

from src import utils
from src.detect.onset_utils import OnsetMaker, bandpass_filter

In [33]:
corp = utils.CorpusMaker.from_excel('corpus_updated', only_annotated=True, only_30_corpus=False).tracks

In [24]:
def proc(item, instr, lp, hp):
    om = OnsetMaker(item, skip_processing=True)
    om.audio = {}
    y, _ = librosa.load(
        path=om._get_channel_override_fpath(instr, rf'{om.data_dir}/processed/mvsep_audio/{om.item["fname"]}_{instr}.{utils.AUDIO_FILE_FMT}',),
        sr=utils.SAMPLE_RATE,
        mono=True,
        offset=0,
        duration=None,
        dtype=np.float64,
        res_type='soxr_vhq',
    )
    om.audio[instr] = bandpass_filter(
        audio=y,
        lowcut=lp,
        highcut=hp,
        order=om.order
    )
    ons = om.onset_detect_cnn(instr)
    gt = rf'{utils.get_project_root()}/references/manual_annotation/{om.item["fname"]}_{instr}.txt'
    return om.compare_onset_detection_accuracy(onsets=ons, fname=gt)

In [25]:
with Parallel(n_jobs=-1, verbose=10) as par:
    bres = par(delayed(proc)(i, 'bass', 30, 494) for i in corp)
    dres = par(delayed(proc)(i, 'drums', 2000, 11000) for i in corp)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  34 | elapsed:    2.1s remaining:   22.2s
[Parallel(n_jobs=-1)]: Done   7 out of  34 | elapsed:    4.0s remaining:   15.3s
[Parallel(n_jobs=-1)]: Done  11 out of  34 | elapsed:    6.6s remaining:   13.7s
[Parallel(n_jobs=-1)]: Done  15 out of  34 | elapsed:    7.7s remaining:    9.7s
[Parallel(n_jobs=-1)]: Done  19 out of  34 | elapsed:    8.8s remaining:    7.0s
[Parallel(n_jobs=-1)]: Done  23 out of  34 | elapsed:    9.4s remaining:    4.5s
[Parallel(n_jobs=-1)]: Done  27 out of  34 | elapsed:    9.6s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done  31 out of  34 | elapsed:   10.2s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  34 out of  34 | elapsed:   11.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  34 | elapsed:    2.1s remaining:   21.5s
[Parallel(n_jobs=-1)]: Done   7 out of  34 | e

In [30]:
print(np.mean([i['f_score'] for i in bres]))
print(np.mean([i['f_score'] for i in dres]))

0.8417224952854624
0.7434682090473006


In [31]:
print(np.std([i['f_score'] for i in bres]))
print(np.std([i['f_score'] for i in dres]))

0.09173451948912655
0.16104497282188715


In [39]:
corp = utils.CorpusMaker.from_excel('corpus_updated', only_annotated=False, only_30_corpus=False).tracks

In [51]:
[i for i in corp if 'brush' in i['notes'].lower()]

[{'track_name': 'Naima',
  'album_name': 'McCoy Tyner plays John Coltrane Live at the Village Vanguard',
  'recording_year': '1997',
  'in_30_corpus': True,
  'bandleader': 'McCoy Tyner',
  'pianist': 'McCoy Tyner',
  'channel_overrides': {},
  'mbz_id': '5439f74b-c5f7-4526-b3cd-da602514e4b9',
  'notes': 'starts after drummer switches to sticks from brushes',
  'time_signature': 4,
  'first_downbeat': 20.53,
  'rating_bass_audio': nan,
  'rating_bass_detection': nan,
  'rating_drums_audio': nan,
  'rating_drums_detection': nan,
  'rating_mix': nan,
  'rating_piano_audio': nan,
  'rating_piano_detection': nan,
  'rating_comments': nan,
  'has_annotations': False,
  'links': {'external': ['https://www.youtube.com/watch?v=oyNKICuqSJ0']},
  'excerpt_duration': '02:23',
  'timestamps': {'start': '03:47', 'end': '06:10'},
  'log': [],
  'musicians': {'pianist': 'McCoy Tyner',
   'bassist': 'George Mraz',
   'drummer': 'Al Foster',
   'leader': 'pianist'},
  'photos': {'musicians': {'pianist'

In [48]:
df = pd.DataFrame(corp)
df['excerpt_duration'] = pd.to_timedelta('00:' + df['excerpt_duration'])
df['recording_year'] = df['recording_year'].astype(int)
df.sort_values(by='excerpt_duration', ascending=False).head(20)

,track_name,album_name,recording_year,in_30_corpus,bandleader,pianist,channel_overrides,mbz_id,notes,time_signature,first_downbeat,rating_bass_audio,rating_bass_detection,rating_drums_audio,rating_drums_detection,rating_mix,rating_piano_audio,rating_piano_detection,rating_comments,has_annotations,links,excerpt_duration,timestamps,log,musicians,photos,fname
1281,Reets and I,At the Golden Circle Volume 4,1962,False,Bud Powell,Bud Powell,{},c04edcb9-e573-454b-9120-a0d3c21d3dc1,,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': ['https://www.youtube.com/watch?v...,0 days 00:07:17,"{'start': '00:42', 'end': '07:59'}",[],"{'pianist': 'Bud Powell', 'bassist': 'Torbjorn...","{'musicians': {'pianist': None, 'bassist': Non...",powellb-reetsandi-hultcrantztspangbergs-1962-c...
168,CTA,Brad Mehldau Trio Live,2006,False,Brad Mehldau,Brad Mehldau,{},9cdbc460-8b30-4c97-9b9c-97637d801b46,,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': ['https://www.youtube.com/watch?v...,0 days 00:07:02,"{'start': '03:48', 'end': '10:50'}",[],"{'pianist': 'Brad Mehldau', 'bassist': 'Larry ...","{'musicians': {'pianist': None, 'bassist': Non...",mehldaub-cta-grenadierlballardj-2006-9cdbc460
67,My Melancholy Baby,Lennie Tristano Personal Recordings 19461970,1956,False,Lennie Tristano,Lennie Tristano,{},039f6f52-1ec4-490b-af96-049aa462e36a,,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': ['https://www.youtube.com/watch?v...,0 days 00:07:00,"{'start': '06:12', 'end': '13:12'}",[],"{'pianist': 'Lennie Tristano', 'bassist': 'Pet...","{'musicians': {'pianist': None, 'bassist': Non...",tristanol-mymelancholybaby-indpweyburnt-1956-0...
375,Blue Hamp Blue Bird,Live at the Jazz Showcase in Chicago Volume Two,1973,False,Hampton Hawes,Hampton Hawes,{},c54c5497-3eb9-4c60-91a2-2be274556eba,,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': ['https://www.youtube.com/watch?v...,0 days 00:06:52,"{'start': '02:17', 'end': '09:09'}",[],"{'pianist': 'Hampton Hawes', 'bassist': 'Cecil...","{'musicians': {'pianist': None, 'bassist': Non...",hawesh-bluehampbluebird-mcbeechaynesr-1973-c54...
546,Well You Neednt,The Perfect Set,1996,True,Kenny Barron,Kenny Barron,{},fbe7467b-27b4-4bbe-8c7f-4b5b9fa1e9b0,,4,10.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': ['https://www.youtube.com/watch?v...,0 days 00:06:51,"{'start': '00:29', 'end': '07:20'}",[],"{'pianist': 'Kenny Barron', 'bassist': 'Ray Dr...","{'musicians': {'pianist': None, 'bassist': Non...",barronk-wellyouneednt-drummondrrileyb-1996-fbe...
1067,If I Were a Bell,Up for It,2001,False,Keith Jarrett,Keith Jarrett,{},a567c076-b8d6-44d3-bf35-3d987023c790,,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': ['https://www.youtube.com/watch?v...,0 days 00:06:39,"{'start': '01:33', 'end': '08:12'}",[],"{'pianist': 'Keith Jarrett', 'bassist': 'Gary ...","{'musicians': {'pianist': None, 'bassist': Non...",jarrettk-ifiwereabell-peacockgdejohnettej-2001...
272,Turnaround When Will the Blues Leave,The Montreal Tapes,1989,False,Paul Bley,Paul Bley,{},31162f56-459c-4d21-aa26-c5071896813c,,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': ['https://www.youtube.com/watch?v...,0 days 00:06:19,"{'start': '01:07', 'end': '07:26'}",[],"{'pianist': 'Paul Bley', 'bassist': 'Charlie H...","{'musicians': {'pianist': None, 'bassist': Non...",bleyp-turnaroundwhenwillthe-hadencmotianp-1989...
544,You Dont Know What Love Is,The Perfect Set,1996,True,Kenny Barron,Kenny Barron,{},15956096-2e6d-4ff8-b2a4-77a2e0783a99,,4,9.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': ['https://www.youtube.com/watch?v...,0 days 00:06:15,"{'start': '03:48', 'end': '10:03'}",[],"{'pianist': 'Kenny Barron', 'bassist': 'Ray Dr...","{'musicians': {'pianist': None, 'bassist': Non...",barronk-youdontknowwhatlove-drummondrrileyb-19...
1280,Blues in the Closet,At the Golden Circle Volume 4,1962,False,Bud Powell,Bud Powell,{},05a04a2a-29ee-4871-a0a0-60f3aed9e3de,,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': ['https://

In [44]:
df[df['recording_year'] < 1956]['excerpt_duration'].describe()

count                          115
mean     0 days 00:01:49.895652173
std      0 days 00:00:45.911121978
min                0 days 00:00:30
25%         0 days 00:01:18.500000
50%                0 days 00:01:40
75%         0 days 00:02:07.500000
max                0 days 00:04:47
Name: excerpt_duration, dtype: object

In [45]:
df[df['recording_year'] >= 1956]['excerpt_duration'].describe()

count                         1198
mean     0 days 00:02:04.656928213
std      0 days 00:01:09.387341587
min                0 days 00:00:19
25%                0 days 00:01:15
50%                0 days 00:01:52
75%                0 days 00:02:40
max                0 days 00:07:17
Name: excerpt_duration, dtype: object

In [52]:
corp = utils.load_corpus_from_files(f'{utils.get_project_root()}/data/cambridge-jazz-trio-database-v02')

In [61]:
oe = [i.item['validation']['bass']['f_score'] for i in corp if i.item['validation']['bass'] is not None]
print(len(oe), np.mean(oe), np.std(oe))

34 0.9168594883058888 0.0803950605204505


In [66]:
oe = [i.item['validation']['bass']['recall'] for i in corp if i.item['validation']['bass'] is not None and 'Sweet Georgia' not in i.item['track_name']]
print(np.min(oe))

0.6276223776223776


In [64]:
oe = [i.item['validation']['bass'] for i in corp if i.item['validation']['bass'] is not None and 'Sweet Georgia' in i.item['track_name']]
oe

[{'f_score': 0.537190082644628,
  'precision': 0.9154929577464789,
  'recall': 0.38011695906432746,
  'mean_asynchrony': 0.001488400476922724,
  'fraction_matched': 0.38011695906432746}]

In [70]:
for i in corp:
    if i.item['validation']['bass'] is not None:
        print(i.item['track_name'], i.item['validation']['bass']['f_score'])

Glorias Step take 1 interrupted 0.9547738693467337
Lil Darlin 0.9473684210526315
Sweet Georgia Brown 0.537190082644628
Between the Devil and the Deep Blue Sea 0.8811630847029077
Back Home Again in Indiana 0.9299719887955181
Dolphin Dance 0.9455445544554454
Take the Coltrane 0.9488139825218476
Ralphs New Blues 0.9518716577540107
Spring Will Be a Little Late This Year 0.924731182795699
All of You 0.8830188679245282
Shuffle Boil 0.9148580968280468
All the Things You Are 0.9396051103368177
Tristeza 0.9533417402269863
Holy Mama 0.9584438549955792
Emily 0.9029850746268656
Denzils Best 0.9449541284403671
SeeSaw 0.9309576837416481
With Malice Towards None 0.8840864440078585
Expectations 0.9366786140979689
Glorias Step take 3 0.9693877551020408
Rifftide 0.9553656220322888
Broadway 0.8416169648774022
Birdsong 0.9677784058790277
Just Friends 0.974477958236659
The Man I Love 0.9388646288209607
Like Someone in Love 0.945017182130584
TraneLike 0.8814589665653495
Someday My Prince Will Come 0.9270633